<!-- ---
title: "Gradient Descent"
format: 
  revealjs:
    theme: custom.scss
    css: custom.css
    width: 1920
    height: 1080
    menu:
      side: right
      width: wide
    template-partials:
      - title-slide.html
    slide-number: c/t
    logo: "eclipse_logo_small.png"
    highlight-style: a11y
    incremental: false
    background-transition: fade
    footer: "©Philipp Pelz - FAU Erlangen-Nürnberg - Data Science for Electron Microscopy"
execute:
  eval: true
  echo: true
--- -->

## Introduction

- Gradient descent is fundamental to understanding optimization
- Key concepts apply to more advanced algorithms
- Important considerations:
  - Learning rate selection
  - Divergence issues
  - Preconditioning techniques

## One-Dimensional Gradient Descent

### Mathematical Foundation

- For continuously differentiable $f: \mathbb{R} \rightarrow \mathbb{R}$
- Taylor expansion:
  $$f(x + \epsilon) = f(x) + \epsilon f'(x) + \mathcal{O}(\epsilon^2)$$
- Moving in negative gradient direction:
  - Choose $\epsilon = -\eta f'(x)$
  - Fixed step size $\eta > 0$
  - Results in: $f(x - \eta f'(x)) \lessapprox f(x)$

---

### Implementation


In [ ]:
#| label: setup2
%matplotlib inline
import d2l
import numpy as np
import torch

def f(x):  # Objective function
    return x ** 2

def f_grad(x):  # Gradient (derivative) of the objective function
    return 2 * x

### Basic Gradient Descent


In [ ]:
#| label: gd-implementation
def gd(eta, f_grad):
    x = 10.0
    results = [x]
    for i in range(10):
        x -= eta * f_grad(x)
        results.append(float(x))
    print(f'epoch 10, x: {x:f}')
    return results

results = gd(0.2, f_grad)

---

### Visualization


In [ ]:
#| label: gd-visualization
def show_trace(results, f):
    n = max(abs(min(results)), abs(max(results)))
    f_line = d2l.arange(-n, n, 0.01)
    d2l.set_figsize()
    d2l.plot([f_line, results], [[f(x) for x in f_line], [
        f(x) for x in results]], 'x', 'f(x)', fmts=['-', '-o'])

show_trace(results, f)

## Learning Rate Effects

### Too Small Learning Rate

- Slow convergence
- More iterations needed
- Example with $\eta = 0.05$:


In [ ]:
#| label: small-lr
show_trace(gd(0.05, f_grad), f)

---

### Too Large Learning Rate

- Solution oscillates
- May diverge
- Example with $\eta = 1.1$:


In [ ]:
#| label: large-lr
show_trace(gd(1.1, f_grad), f)

---

### Local Minima

- Nonconvex functions have multiple minima
- Example: $f(x) = x \cdot \cos(cx)$
- High learning rates can lead to poor local minima


In [ ]:
#| label: local-minima
c = d2l.tensor(0.15 * np.pi)

def f(x):  # Objective function
    return x * d2l.cos(c * x)

def f_grad(x):  # Gradient of the objective function
    return d2l.cos(c * x) - c * x * d2l.sin(c * x)

show_trace(gd(2, f_grad), f)

## Multivariate Gradient Descent

### Mathematical Foundation

- For $f: \mathbb{R}^d \to \mathbb{R}$
- Gradient vector: $\nabla f(\mathbf{x}) = [\frac{\partial f(\mathbf{x})}{\partial x_1}, \ldots, \frac{\partial f(\mathbf{x})}{\partial x_d}]^\top$
- Taylor expansion:
  $$f(\mathbf{x} + \boldsymbol{\epsilon}) = f(\mathbf{x}) + \mathbf{\boldsymbol{\epsilon}}^\top \nabla f(\mathbf{x}) + \mathcal{O}(\|\boldsymbol{\epsilon}\|^2)$$
- Update rule: $\mathbf{x} \leftarrow \mathbf{x} - \eta \nabla f(\mathbf{x})$

---

### Implementation


In [ ]:
#| label: multivariate-gd
def train_2d(trainer, steps=20, f_grad=None):
    """Optimize a 2D objective function with a customized trainer."""
    x1, x2, s1, s2 = -5, -2, 0, 0
    results = [(x1, x2)]
    for i in range(steps):
        if f_grad:
            x1, x2, s1, s2 = trainer(x1, x2, s1, s2, f_grad)
        else:
            x1, x2, s1, s2 = trainer(x1, x2, s1, s2)
        results.append((x1, x2))
    print(f'epoch {i + 1}, x1: {float(x1):f}, x2: {float(x2):f}')
    return results

def show_trace_2d(f, results):
    """Show the trace of 2D variables during optimization."""
    d2l.set_figsize()
    d2l.plt.plot(*zip(*results), '-o', color='#ff7f0e')
    x1, x2 = d2l.meshgrid(d2l.arange(-5.5, 1.0, 0.1),
                          d2l.arange(-3.0, 1.0, 0.1), indexing='ij')
    d2l.plt.contour(x1, x2, f(x1, x2), colors='#1f77b4')
    d2l.plt.xlabel('x1')
    d2l.plt.ylabel('x2')

---

### Example: Quadratic Function


In [ ]:
#| label: quadratic-example
def f_2d(x1, x2):  # Objective function
    return x1 ** 2 + 2 * x2 ** 2

def f_2d_grad(x1, x2):  # Gradient of the objective function
    return (2 * x1, 4 * x2)

def gd_2d(x1, x2, s1, s2, f_grad):
    g1, g2 = f_grad(x1, x2)
    return (x1 - eta * g1, x2 - eta * g2, 0, 0)

eta = 0.1
show_trace_2d(f_2d, train_2d(gd_2d, f_grad=f_2d_grad))

## Adaptive Methods

### Newton's Method

- Uses second-order information
- Taylor expansion with Hessian:
  $$f(\mathbf{x} + \boldsymbol{\epsilon}) = f(\mathbf{x}) + \boldsymbol{\epsilon}^\top \nabla f(\mathbf{x}) + \frac{1}{2} \boldsymbol{\epsilon}^\top \nabla^2 f(\mathbf{x}) \boldsymbol{\epsilon} + \mathcal{O}(\|\boldsymbol{\epsilon}\|^3)$$
- Update rule: $\boldsymbol{\epsilon} = -\mathbf{H}^{-1} \nabla f(\mathbf{x})$

---


### Implementation


In [ ]:
#| label: newton-method
c = d2l.tensor(0.5)

def f(x):  # Objective function
    return d2l.cosh(c * x)

def f_grad(x):  # Gradient of the objective function
    return c * d2l.sinh(c * x)

def f_hess(x):  # Hessian of the objective function
    return c**2 * d2l.cosh(c * x)

def newton(eta=1):
    x = 10.0
    results = [x]
    for i in range(10):
        x -= eta * f_grad(x) / f_hess(x)
        results.append(float(x))
    print('epoch 10, x:', x)
    return results

show_trace(newton(), f)

---


### Nonconvex Example


In [ ]:
#| label: nonconvex-newton
c = d2l.tensor(0.15 * np.pi)

def f(x):  # Objective function
    return x * d2l.cos(c * x)

def f_grad(x):  # Gradient of the objective function
    return d2l.cos(c * x) - c * x * d2l.sin(c * x)

def f_hess(x):  # Hessian of the objective function
    return - 2 * c * d2l.sin(c * x) - x * c**2 * d2l.cos(c * x)

show_trace(newton(0.5), f)

## Preconditioning

### Key Concepts

- Avoid full Hessian computation
- Use diagonal entries only
- Update rule: $\mathbf{x} \leftarrow \mathbf{x} - \eta \textrm{diag}(\mathbf{H})^{-1} \nabla f(\mathbf{x})$
- Benefits:
  - Different learning rates per variable
  - Handles scale mismatches
  - More efficient than full Newton's method

## Summary

- Learning rate selection is crucial
- Local minima can trap gradient descent
- High dimensions require careful learning rate adjustment
- Preconditioning helps with scale issues
- Newton's method:
  - Fast convergence for convex problems
  - Requires careful handling for nonconvex problems
  - Computationally expensive for large problems

## Exercises

1. Experiment with different learning rates and objective functions
2. Implement line search for convex optimization
3. Design a slow-converging 2D objective function
4. Implement lightweight Newton's method with preconditioning
5. Test algorithms on rotated coordinate systems 